In [1]:
# imports

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [2]:
# import load_data function from loadData.py
from loadData import load_data

train_data, train_labels, test_data, test_labels = load_data()

# type of train_data
print(type(train_data))

Loading data
Data loaded
<class 'torch.Tensor'>


In [3]:
# Custom dataset class
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.labels[index]
        return x, y


In [15]:
# define the network class

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = torch.flatten(x, -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
# define the training function

def train(model, train_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        for i, data in enumerate(train_loader):
            i, labels = data
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            if i % 100 == 0:
                print(f"Epoch {epoch} Batch {i} Loss: {loss.item()}")
                
    print("Finished Training")

In [6]:
# define the testing function

def test(model, test_loader):
    correct = 0
    total = 0

    with torch.no_grad():
        for data, labels in test_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    print(f"Accuracy: {correct / total}")

In [19]:
# data loader
train_dataset = [train_data, train_labels]
test_dataset = (test_data, test_labels)

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=True)

In [16]:
# create an instance of the network
model = Net()

# define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# run one forward and backward pass on the first item in the train_loader
model.forward(train_data[0])


tensor([-0.1371, -0.0044, -0.0037, -0.0116,  0.0398, -0.0253, -0.1528,  0.1145,
        -0.0442,  0.0861], grad_fn=<AddBackward0>)

In [20]:
# train the network
train(model, train_loader, criterion, optimizer, 5)

# test the network
test(model, test_loader)

RuntimeError: stack expects each tensor to be equal size, but got [60000] at entry 0 and [60000, 784] at entry 1